In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

In [3]:
df_aaindex = pd.read_csv('data/aaindex/df_aaindex19.csv')
print(df_aaindex.shape)
df_aaindex.head(1)
tmp = df_aaindex.drop('Unnamed: 0',axis=1).T
aa2val = dict()
for aa, val in zip(tmp.index, tmp.values):
    aa2val[aa]=val

(19, 21)


In [4]:
df_detect_peptide_train = pd.read_csv('data/df_detect_peptide_train.csv')
test = pd.read_csv('data/df_detect_peptide_test.csv')
train, val = train_test_split(df_detect_peptide_train, test_size=0.2, random_state=7)

In [5]:
train.head(1)

,peptide,En,Ec,E1,E2,protein,PEP,ID
595411,K.QELNEPPKQSTSFLVLQEILESEEKGDPNK.P,VYKMLQEKQELNEPP,EEKGDPNKPSGFRSV,QELNEPPKQSTSFLV,EILESEEKGDPNKPS,sp|O00151|PDLI1_HUMAN,QELNEPPKQSTSFLVLQEILESEEKGDPNK,0


In [6]:
def get_data_labelEnc(df):
    label_enc = {v:k+1 for k, v in enumerate('ARNDCQEGHILKMFPSTWYV')}
    label_enc['Z']=0
    aa_data = [np.array([seq.count(a) for a in 'ARNDCQEGHILKMFPSTWYV'] + list(np.array([aa2val[aa] for aa in seq]).sum(axis=0)))
               for seq in df.PEP.values]
    pep_data = [[label_enc[aa] for aa in seq] + [0]*(30-len(seq))
               for seq in df.PEP.values]
    en_data = [[label_enc[aa] for aa in seq]
               for seq in df.En.values]
    ec_data = [[label_enc[aa] for aa in seq]
               for seq in df.Ec.values]
    e1_data = [[label_enc[aa] for aa in seq]
               if seq != '-' else [0 for _ in range(15)]
               for seq in df.E1.values]
    e2_data = [[label_enc[aa] for aa in seq]
               if seq != '-' else [0 for _ in range(15)]
               for seq in df.E2.values]
    return np.array(aa_data), np.array(pep_data), np.array(en_data), np.array(ec_data), np.array(e1_data), np.array(e2_data), np.array(df.ID.values)

In [34]:
aa_train, pep_train, en_train, ec_train, e1_train, e2_train, y_train = get_data_labelEnc(train)
aa_val, pep_val, en_val, ec_val, e1_val, e2_val, y_val = get_data_labelEnc(val)
aa_test, pep_test, en_test, ec_test, e1_test, e2_test, y_test = get_data_labelEnc(test)
print(aa_train.shape, pep_train.shape, en_train.shape, ec_train.shape, e1_train.shape, e2_train.shape, y_train.shape)
print(aa_val.shape, pep_val.shape, en_val.shape, ec_val.shape, e1_val.shape, e2_val.shape, y_val.shape)
print(aa_test.shape, pep_test.shape, en_test.shape, ec_test.shape, e1_test.shape, e2_test.shape, y_test.shape)

(543516, 39) (543516, 30) (543516, 15) (543516, 15) (543516, 15) (543516, 15) (543516,)
(135880, 39) (135880, 30) (135880, 15) (135880, 15) (135880, 15) (135880, 15) (135880,)
(133992, 39) (133992, 30) (133992, 15) (133992, 15) (133992, 15) (133992, 15) (133992,)


In [8]:
y_train = np.array([[1, 1, 0, 0] + [_] for _ in y_train])
y_val = np.array([[1, 1, 0, 0] + [_] for _ in y_val])
y_test = np.array([[1, 1, 0, 0] + [_] for _ in y_test])

In [9]:
print(y_train.shape, y_val.shape, y_test.shape)

(543516, 5) (135880, 5) (133992, 5)


# Train

In [10]:
from tensorflow.python.client import device_lib

import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_virtual_device_configuration(gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024*8)])
    except RuntimeError as e:
        print(e)

In [11]:
def plot_graphs(history, metric):
    plt.plot(history.history[metric])
    plt.plot(history.history['val_'+metric], '')
    plt.xlabel("Epochs")
    plt.ylabel(metric)
    plt.legend([metric, 'val_'+metric])

# high param

In [12]:
from tensorflow.keras import layers
from tensorflow import keras

In [13]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)
    
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

In [14]:
vocab_size = 21  # 20 Amino acid + zero padding
maxlen_pep = 30
maxlen_ts = 15

pep_embed_dim = 128  # Embedding size for each token
pep_num_heads = 4  # Number of attention heads
pep_ff_dim = 64  # Hidden layer size in feed forward network inside transformer

ts_embed_dim = 64  # Embedding size for each token
ts_num_heads = 4  # Number of attention heads
ts_ff_dim = 32  # Hidden layer size in feed forward network inside transformer

In [65]:
embedding_layer_pep = TokenAndPositionEmbedding(maxlen_pep, vocab_size, pep_embed_dim)
transformer_block_pep = TransformerBlock(pep_embed_dim, pep_num_heads, pep_ff_dim)
embedding_layer_ts = TokenAndPositionEmbedding(maxlen_ts, vocab_size, ts_embed_dim)
transformer_block_ts = TransformerBlock(ts_embed_dim, ts_num_heads, ts_ff_dim)

# peptide embedding
pep_input = layers.Input(shape=(maxlen_pep,), name='pep_input')
pep_embed = embedding_layer_pep(pep_input)
pep_embed = transformer_block_pep(pep_embed)

# peptide info
meta_input = tf.keras.layers.Input(shape=((39,)))  

# cleavage site embedding
n_input = layers.Input(shape=(maxlen_ts,), name='n_input')
n_embed = embedding_layer_ts(n_input)
n_embed = transformer_block_ts(n_embed)

c_input = layers.Input(shape=(maxlen_ts,), name='c_input')
c_embed = embedding_layer_ts(c_input)
c_embed = transformer_block_ts(c_embed)

m1_input = layers.Input(shape=(maxlen_ts,), name='m1_input')
m1_embed = embedding_layer_ts(m1_input)
m1_embed = transformer_block_ts(m1_embed)

m2_input = layers.Input(shape=(maxlen_ts,), name='m2_input')
m2_embed = embedding_layer_ts(m2_input)
m2_embed = transformer_block_ts(m2_embed)

# peptide CNN + LSTM
p_lstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16), name='pep_lstm')
# pep_reshape = keras.backend.reshape(pep_embed, [-1, 30, 128, 1], name='pep_reshape')
# pep_cnn = tf.keras.layers.TimeDistributed(tf.keras.layers.Conv1D(filters=32, kernel_size=2, strides=1, padding='same', name='pep_cnn'),
#                                          name='pep_cnn_t')(pep_reshape)
# pep_pool = tf.keras.layers.TimeDistributed(tf.keras.layers.AveragePooling1D(pool_size=2, name='pep_pool'), name='pep_pool_t')(pep_cnn)
# pep_flat = tf.keras.layers.TimeDistributed(tf.keras.layers.Flatten(name='pep_flat'), name='pep_flat_t')(pep_pool)
# pep_lstm = p_lstm(pep_flat)
pep_lstm = p_lstm(pep_embed)
pep_lstm = tf.keras.layers.Dense(32, activation='relu', name='pep_dense')(pep_lstm)
pep_lstm = tf.keras.layers.Dropout(np.random.uniform(0, 0.2))(pep_lstm)

# meta info
meta = tf.keras.layers.Dense(32, activation='relu', name='meta_dense')(meta_input)
meta = tf.keras.layers.Dropout(np.random.uniform(0, 0.2))(meta)

# cleavage site CNN+LSTM
ts_lstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16), name='ts_lstm')
ts_flat = tf.keras.layers.Dense(16, activation='relu', name='ts_flat')
ts_drop = tf.keras.layers.Dropout(np.random.uniform(0, 0.2))

# n_reshape = keras.backend.reshape(n_embed, [-1, 15, 64, 1], name='n_reshape')
# n_cnn = tf.keras.layers.TimeDistributed(tf.keras.layers.Conv1D(filters=16, kernel_size=2, strides=1, padding='same'))(n_reshape)
# n_pool = tf.keras.layers.TimeDistributed(tf.keras.layers.AveragePooling1D(pool_size=2))(n_cnn)
# n_flat = tf.keras.layers.TimeDistributed(tf.keras.layers.Flatten())(n_pool)
# n_lstm = ts_lstm(n_flat)
# n_lstm = tf.keras.layers.Dense(16, activation='relu', name='n_dense')(n_lstm)
# n_lstm = tf.keras.layers.Dropout(np.random.uniform(0, 0.2))(n_lstm)
n_lstm = ts_lstm(n_embed)
n_lstm = ts_flat(n_lstm)
n_lstm = ts_drop(n_lstm)

# c_reshape = keras.backend.reshape(c_embed, [-1, 15, 64, 1], name='c_reshape')
# c_cnn = tf.keras.layers.TimeDistributed(tf.keras.layers.Conv1D(filters=16, kernel_size=2, strides=1, padding='same'))(c_reshape)
# c_pool = tf.keras.layers.TimeDistributed(tf.keras.layers.AveragePooling1D(pool_size=2))(c_cnn)
# c_flat = tf.keras.layers.TimeDistributed(tf.keras.layers.Flatten())(c_pool)
# c_lstm = ts_lstm(c_flat)
# c_lstm = tf.keras.layers.Dense(16, activation='relu', name='c_dense')(c_lstm)
# c_lstm = tf.keras.layers.Dropout(np.random.uniform(0, 0.2))(c_lstm)
c_lstm = ts_lstm(c_embed)
c_lstm = ts_flat(c_lstm)
c_lstm = ts_drop(c_lstm)

# m1_reshape = keras.backend.reshape(m1_embed, [-1, 15, 64, 1], name='m1_reshape')
# m1_cnn = tf.keras.layers.TimeDistributed(tf.keras.layers.Conv1D(filters=16, kernel_size=2, strides=1, padding='same'))(m1_reshape)
# m1_pool = tf.keras.layers.TimeDistributed(tf.keras.layers.AveragePooling1D(pool_size=2))(m1_cnn)
# m1_flat = tf.keras.layers.TimeDistributed(tf.keras.layers.Flatten())(m1_pool)
# m1_lstm = ts_lstm(m1_flat)
# m1_lstm = tf.keras.layers.Dense(16, activation='relu', name='m1_dense')(m1_lstm)
# m1_lstm = tf.keras.layers.Dropout(np.random.uniform(0, 0.2))(m1_lstm)
m1_lstm = ts_lstm(m1_embed)
m1_lstm = ts_flat(m1_lstm)
m1_lstm = ts_drop(m1_lstm)

# m2_reshape = keras.backend.reshape(m2_embed, [-1, 15, 64, 1], name='m2_reshape')
# m2_cnn = tf.keras.layers.TimeDistributed(tf.keras.layers.Conv1D(filters=16, kernel_size=2, strides=1, padding='same'))(m2_reshape)
# m2_pool = tf.keras.layers.TimeDistributed(tf.keras.layers.AveragePooling1D(pool_size=2))(m2_cnn)
# m2_flat = tf.keras.layers.TimeDistributed(tf.keras.layers.Flatten())(m2_pool)
# m2_lstm = ts_lstm(m2_flat)
# m2_lstm = tf.keras.layers.Dense(16, activation='relu', name='m2_dense')(m2_lstm)
# m2_lstm = tf.keras.layers.Dropout(np.random.uniform(0, 0.2))(m2_lstm)
m2_lstm = ts_lstm(m2_embed)
m2_lstm = ts_flat(m2_lstm)
m2_lstm = ts_drop(m2_lstm)

# Digestibility 
output_array = [] 
metrics_array = {}
loss_array = {}
digestibility_flat = tf.keras.layers.Dense(1, activation='sigmoid', name="digestibility_output")

for i, dense_layer in enumerate([n_lstm, c_lstm, m1_lstm, m2_lstm]):
    layer = tf.keras.layers.concatenate([pep_lstm, dense_layer])
    # A Dense Layer is created for each output
    digestibility_output = digestibility_flat(layer)
    output_array.append(digestibility_output)
    if i ==0:
        metrics_array["digestibility_output"] = 'binary_accuracy'
        loss_array["digestibility_output"] = 'binary_crossentropy'
    else:
        metrics_array["digestibility_output_"+str(i)] = 'binary_accuracy'
        loss_array["digestibility_output_"+str(i)] = 'binary_crossentropy'


# Detectability
detectability_output = tf.keras.layers.concatenate([pep_lstm, 
                                                     meta,
                                                     n_lstm,
                                                     c_lstm,
                                                     m1_lstm,
                                                     m2_lstm])

detectability_output = tf.keras.layers.Dense(64, activation='relu', name='detect_out1')(detectability_output)
detectability_output = tf.keras.layers.Dropout(np.random.uniform(0, 0.2), name='detect_drop1')(detectability_output)
detectability_output = tf.keras.layers.Dense(32, activation='relu', name='detect_out2')(detectability_output)
detectability_output = tf.keras.layers.Dropout(np.random.uniform(0, 0.2), name='detect_drop2')(detectability_output)
detectability_output = tf.keras.layers.Dense(1, activation = 'sigmoid', name='detectability_output')(detectability_output)

output_array.append(detectability_output)

metrics_array['detectability_output'] = 'binary_accuracy'
loss_array['detectability_output'] = 'binary_crossentropy'

In [66]:
metrics_array, loss_array, output_array

({'digestibility_output': 'binary_accuracy',
  'digestibility_output_1': 'binary_accuracy',
  'digestibility_output_2': 'binary_accuracy',
  'digestibility_output_3': 'binary_accuracy',
  'detectability_output': 'binary_accuracy'},
 {'digestibility_output': 'binary_crossentropy',
  'digestibility_output_1': 'binary_crossentropy',
  'digestibility_output_2': 'binary_crossentropy',
  'digestibility_output_3': 'binary_crossentropy',
  'detectability_output': 'binary_crossentropy'},
 [<KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'digestibility_output')>,
  <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'digestibility_output')>,
  <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'digestibility_output')>,
  <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'digestibility_output')>,
  <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'detectability_output')>])

In [67]:
model_high = tf.keras.Model(inputs=[pep_input, meta_input, n_input, c_input, m1_input, m2_input],
                            outputs=output_array)
model_high.summary()

model_high.compile(loss=loss_array,
                   optimizer=tf.keras.optimizers.Adam(1e-4),
                   metrics=metrics_array)

es = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                      mode='min', 
                                      verbose=1,
                                      patience=50)

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
pep_input (InputLayer)          [(None, 30)]         0                                            
__________________________________________________________________________________________________
n_input (InputLayer)            [(None, 15)]         0                                            
__________________________________________________________________________________________________
c_input (InputLayer)            [(None, 15)]         0                                            
__________________________________________________________________________________________________
m1_input (InputLayer)           [(None, 15)]         0                                            
____________________________________________________________________________________________

In [68]:
history = model_high.fit(
            [pep_train, aa_train, en_train, ec_train, e1_train, e2_train],
            [np.array([1]*len(y_train)), np.array([1]*len(y_train)), np.array([0]*len(y_train)), np.array([0]*len(y_train)), y_train], 
            epochs=100,
            batch_size=256,
            validation_data=(
            [pep_val, aa_val, en_val, ec_val, e1_val, e2_val],
            [np.array([1]*len(y_val)), np.array([1]*len(y_val)), np.array([0]*len(y_val)), np.array([0]*len(y_val)), y_val]),
#             callbacks=[es]
)

Epoch 1/300
2124/2124 [==============================] - 106s 43ms/step - loss: 10.8125 - digestibility_output_loss: 0.4579 - digestibility_output_1_loss: 0.4572 - digestibility_output_2_loss: 0.7284 - digestibility_output_3_loss: 0.3454 - detectability_output_loss: 8.8236 - digestibility_output_binary_accuracy: 0.8203 - digestibility_output_1_binary_accuracy: 0.8211 - digestibility_output_2_binary_accuracy: 0.4475 - digestibility_output_3_binary_accuracy: 0.7794 - detectability_output_binary_accuracy: 0.5252 - val_loss: 2.3272 - val_digestibility_output_loss: 0.3839 - val_digestibility_output_1_loss: 0.3837 - val_digestibility_output_2_loss: 0.6566 - val_digestibility_output_3_loss: 0.2288 - val_detectability_output_loss: 0.6742 - val_digestibility_output_binary_accuracy: 0.8233 - val_digestibility_output_1_binary_accuracy: 0.8247 - val_digestibility_output_2_binary_accuracy: 0.4860 - val_digestibility_output_3_binary_accuracy: 0.8704 - val_detectability_output_binary_accuracy: 0.6433

2124/2124 [==============================] - 89s 42ms/step - loss: 2.0489 - digestibility_output_loss: 0.3783 - digestibility_output_1_loss: 0.3810 - digestibility_output_2_loss: 0.6301 - digestibility_output_3_loss: 0.2271 - detectability_output_loss: 0.4324 - digestibility_output_binary_accuracy: 0.8256 - digestibility_output_1_binary_accuracy: 0.8190 - digestibility_output_2_binary_accuracy: 0.4936 - digestibility_output_3_binary_accuracy: 0.8740 - detectability_output_binary_accuracy: 0.8037 - val_loss: 2.0275 - val_digestibility_output_loss: 0.3832 - val_digestibility_output_1_loss: 0.3847 - val_digestibility_output_2_loss: 0.6199 - val_digestibility_output_3_loss: 0.2237 - val_detectability_output_loss: 0.4160 - val_digestibility_output_binary_accuracy: 0.8189 - val_digestibility_output_1_binary_accuracy: 0.8108 - val_digestibility_output_2_binary_accuracy: 0.5006 - val_digestibility_output_3_binary_accuracy: 0.8819 - val_detectability_output_binary_accuracy: 0.8109
Epoch 10/300


2124/2124 [==============================] - 90s 42ms/step - loss: 2.0135 - digestibility_output_loss: 0.3784 - digestibility_output_1_loss: 0.3788 - digestibility_output_2_loss: 0.6290 - digestibility_output_3_loss: 0.2256 - detectability_output_loss: 0.4017 - digestibility_output_binary_accuracy: 0.8173 - digestibility_output_1_binary_accuracy: 0.8160 - digestibility_output_2_binary_accuracy: 0.5038 - digestibility_output_3_binary_accuracy: 0.8758 - detectability_output_binary_accuracy: 0.8197 - val_loss: 2.0034 - val_digestibility_output_loss: 0.3855 - val_digestibility_output_1_loss: 0.3851 - val_digestibility_output_2_loss: 0.6159 - val_digestibility_output_3_loss: 0.2222 - val_detectability_output_loss: 0.3948 - val_digestibility_output_binary_accuracy: 0.7998 - val_digestibility_output_1_binary_accuracy: 0.7985 - val_digestibility_output_2_binary_accuracy: 0.5187 - val_digestibility_output_3_binary_accuracy: 0.8957 - val_detectability_output_binary_accuracy: 0.8225
Epoch 18/300


2124/2124 [==============================] - 89s 42ms/step - loss: 1.9972 - digestibility_output_loss: 0.3777 - digestibility_output_1_loss: 0.3783 - digestibility_output_2_loss: 0.6270 - digestibility_output_3_loss: 0.2254 - detectability_output_loss: 0.3888 - digestibility_output_binary_accuracy: 0.8141 - digestibility_output_1_binary_accuracy: 0.8134 - digestibility_output_2_binary_accuracy: 0.5096 - digestibility_output_3_binary_accuracy: 0.8794 - detectability_output_binary_accuracy: 0.8258 - val_loss: 1.9963 - val_digestibility_output_loss: 0.3736 - val_digestibility_output_1_loss: 0.3751 - val_digestibility_output_2_loss: 0.6303 - val_digestibility_output_3_loss: 0.2276 - val_detectability_output_loss: 0.3896 - val_digestibility_output_binary_accuracy: 0.8311 - val_digestibility_output_1_binary_accuracy: 0.8258 - val_digestibility_output_2_binary_accuracy: 0.4923 - val_digestibility_output_3_binary_accuracy: 0.8645 - val_detectability_output_binary_accuracy: 0.8246
Epoch 26/300


2124/2124 [==============================] - 89s 42ms/step - loss: 1.9894 - digestibility_output_loss: 0.3778 - digestibility_output_1_loss: 0.3784 - digestibility_output_2_loss: 0.6260 - digestibility_output_3_loss: 0.2251 - detectability_output_loss: 0.3821 - digestibility_output_binary_accuracy: 0.8148 - digestibility_output_1_binary_accuracy: 0.8144 - digestibility_output_2_binary_accuracy: 0.5101 - digestibility_output_3_binary_accuracy: 0.8790 - detectability_output_binary_accuracy: 0.8295 - val_loss: 1.9907 - val_digestibility_output_loss: 0.3757 - val_digestibility_output_1_loss: 0.3790 - val_digestibility_output_2_loss: 0.6268 - val_digestibility_output_3_loss: 0.2246 - val_detectability_output_loss: 0.3846 - val_digestibility_output_binary_accuracy: 0.8202 - val_digestibility_output_1_binary_accuracy: 0.8145 - val_digestibility_output_2_binary_accuracy: 0.5045 - val_digestibility_output_3_binary_accuracy: 0.8758 - val_detectability_output_binary_accuracy: 0.8280
Epoch 34/300


...

2124/2124 [==============================] - 90s 43ms/step - loss: 1.9567 - digestibility_output_loss: 0.3767 - digestibility_output_1_loss: 0.3777 - digestibility_output_2_loss: 0.6244 - digestibility_output_3_loss: 0.2236 - detectability_output_loss: 0.3542 - digestibility_output_binary_accuracy: 0.8151 - digestibility_output_1_binary_accuracy: 0.8154 - digestibility_output_2_binary_accuracy: 0.5105 - digestibility_output_3_binary_accuracy: 0.8793 - detectability_output_binary_accuracy: 0.8431 - val_loss: 1.9864 - val_digestibility_output_loss: 0.3753 - val_digestibility_output_1_loss: 0.3764 - val_digestibility_output_2_loss: 0.6280 - val_digestibility_output_3_loss: 0.2253 - val_detectability_output_loss: 0.3814 - val_digestibility_output_binary_accuracy: 0.8170 - val_digestibility_output_1_binary_accuracy: 0.8148 - val_digestibility_output_2_binary_accuracy: 0.5076 - val_digestibility_output_3_binary_accuracy: 0.8767 - val_detectability_output_binary_accuracy: 0.8283
Epoch 98/300


2124/2124 [==============================] - 90s 42ms/step - loss: 1.9529 - digestibility_output_loss: 0.3770 - digestibility_output_1_loss: 0.3775 - digestibility_output_2_loss: 0.6237 - digestibility_output_3_loss: 0.2241 - detectability_output_loss: 0.3505 - digestibility_output_binary_accuracy: 0.8155 - digestibility_output_1_binary_accuracy: 0.8151 - digestibility_output_2_binary_accuracy: 0.5116 - digestibility_output_3_binary_accuracy: 0.8786 - detectability_output_binary_accuracy: 0.8449 - val_loss: 1.9857 - val_digestibility_output_loss: 0.3790 - val_digestibility_output_1_loss: 0.3809 - val_digestibility_output_2_loss: 0.6220 - val_digestibility_output_3_loss: 0.2226 - val_detectability_output_loss: 0.3813 - val_digestibility_output_binary_accuracy: 0.8184 - val_digestibility_output_1_binary_accuracy: 0.8165 - val_digestibility_output_2_binary_accuracy: 0.5061 - val_digestibility_output_3_binary_accuracy: 0.8743 - val_detectability_output_binary_accuracy: 0.8285
Epoch 106/300

2124/2124 [==============================] - 89s 42ms/step - loss: 1.9496 - digestibility_output_loss: 0.3771 - digestibility_output_1_loss: 0.3778 - digestibility_output_2_loss: 0.6227 - digestibility_output_3_loss: 0.2245 - detectability_output_loss: 0.3475 - digestibility_output_binary_accuracy: 0.8147 - digestibility_output_1_binary_accuracy: 0.8161 - digestibility_output_2_binary_accuracy: 0.5115 - digestibility_output_3_binary_accuracy: 0.8787 - detectability_output_binary_accuracy: 0.8472 - val_loss: 1.9869 - val_digestibility_output_loss: 0.3766 - val_digestibility_output_1_loss: 0.3789 - val_digestibility_output_2_loss: 0.6258 - val_digestibility_output_3_loss: 0.2235 - val_detectability_output_loss: 0.3821 - val_digestibility_output_binary_accuracy: 0.8166 - val_digestibility_output_1_binary_accuracy: 0.8129 - val_digestibility_output_2_binary_accuracy: 0.5091 - val_digestibility_output_3_binary_accuracy: 0.8763 - val_detectability_output_binary_accuracy: 0.8293
Epoch 114/300

2124/2124 [==============================] - 89s 42ms/step - loss: 1.9478 - digestibility_output_loss: 0.3770 - digestibility_output_1_loss: 0.3777 - digestibility_output_2_loss: 0.6240 - digestibility_output_3_loss: 0.2242 - detectability_output_loss: 0.3449 - digestibility_output_binary_accuracy: 0.8152 - digestibility_output_1_binary_accuracy: 0.8157 - digestibility_output_2_binary_accuracy: 0.5114 - digestibility_output_3_binary_accuracy: 0.8788 - detectability_output_binary_accuracy: 0.8480 - val_loss: 1.9886 - val_digestibility_output_loss: 0.3773 - val_digestibility_output_1_loss: 0.3786 - val_digestibility_output_2_loss: 0.6254 - val_digestibility_output_3_loss: 0.2241 - val_detectability_output_loss: 0.3832 - val_digestibility_output_binary_accuracy: 0.8191 - val_digestibility_output_1_binary_accuracy: 0.8166 - val_digestibility_output_2_binary_accuracy: 0.5052 - val_digestibility_output_3_binary_accuracy: 0.8726 - val_detectability_output_binary_accuracy: 0.8281
Epoch 122/300

KeyboardInterrupt: 

In [ ]:
plt.figure(figsize=(16,2))
plt.subplot(1,2,1)
plot_graphs(history, 'accuracy')
plt.subplot(1,2,2)
plot_graphs(history, 'loss')

# eval

In [ ]:
test_loss, test_acc = model_high.evaluate([pep_test, aa_test, en_test, ec_test, e1_test, e2_test], y_test)
print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_acc))

# prediction
y_pred = [1 if i>=0.5 else 0 for i in model_high.predict([pep_test, aa_test, en_test, ec_test, e1_test, e2_test])]
print(classification_report(y_test, y_pred))
# AUC
probs = model_high.predict([pep_test, aa_test, en_test, ec_test, e1_test, e2_test])
rf_auc = roc_auc_score(y_test, probs)
print('rf auc : {}'.format(rf_auc))
# plot the roc curve for the model_high
rf_fpr, rf_tpr, _ = roc_curve(y_test, probs)
plt.figure(figsize=(4, 4))
plt.plot(rf_fpr, rf_tpr, marker='.', label='RF AUC = {:.4f}'.format(rf_auc), color='orange')
plt.title('ROC curve')
# axis labels
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
# show the legend
plt.legend()
# show the plot
plt.show()

cf_matrix = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(3, 3))
sns.heatmap(cf_matrix, annot=True, fmt=',.0f')
plt.show()
plt.figure(figsize=(3, 3))
sns.heatmap(cf_matrix/np.sum(cf_matrix), annot=True, 
            fmt='.2%', cmap='Blues')
plt.show()